# Imports

In [1]:
import numpy as np
import pandas as pd
import time
import math
from collections import Counter

# Carregando os dados

In [2]:
dtype={
       "region": object, 
       "city":object, 
       "parent_category_name":object,
       "category_name": object,
       "title":object,
       "description":object, 
       "price":np.float32, 
       "activation_date": object, 
       "user_type": object, 
       "image":object, 
       "image_top_1":np.float32, 
       "deal_probability":np.float32,
       "item_seq_number": np.uint,
       "item_id": object,
       "param_1": object,
       "param_2": object,
       "param_3": object,
       "user_id": object
      }

start = time.time()
df_train = pd.read_csv('./data/train.csv', dtype = dtype, encoding='utf8')
df_test  = pd.read_csv('./data/test.csv' , dtype = dtype, encoding='utf8')
end = time.time()

print('Tempo de carregamento dos CSVs: '+str("%.2f" % (end - start))+'s\n')


text_columns        = ["title","description"]
colunas_categoricas_preenchidas = ["user_type","parent_category_name","category_name", "user_id","image_top_1"]
colunas_numericas_preenchidas   = ["item_seq_number"]

'''
As colunas categóricas parcialmente preenchidas são: param_1, param_2, param_3
A coluna numérica parcialmente preenchida é price
Logo só é necessário se preocupar em preencher-las
'''


matrizes_train_resultado = []
matrizes_test_resultado = []


Tempo de carregamento dos CSVs: 24.88s



# Eliminando colunas desnecessárias

In [3]:
columns_to_drop =['item_id']
df_train.drop(columns=columns_to_drop, inplace=True)
df_test.drop (columns=columns_to_drop, inplace=True)

# Apagando dados de treinamento não preenchidos

In [4]:
df_train.dropna(inplace =True)
#df_train = df_train.head(1000)

# Carregamento das features geradas pelas imagens

In [5]:
# Features calculadas em notebook externo

start = time.time()

# merge dados de treino
#df_train["image"].fillna(value="no-image", inplace=True)
df_images_train = pd.read_csv('./data/train_jpg.csv', encoding='utf8')
df_train  = df_train.merge(df_train.merge(df_images_train, left_on = "image", right_on = "id", copy=False, sort=False),copy=False)
df_images_train = None
df_train.drop(columns=["image","id"],inplace=True)

# merge dados de teste
df_test["image"].fillna(value="no-image", inplace=True)
df_images_test = pd.read_csv('./data/test_jpg.csv', encoding='utf8')
df_test  = df_test.merge(df_test.merge(df_images_test, left_on = "image", right_on = "id", copy=False, sort=False),copy=False)
df_test.drop(columns=["image","id"],inplace=True)

df_images_test.drop(columns=["id"], inplace = True)
numeric_columns_images = df_images_test.columns.tolist()
colunas_numericas_preenchidas += numeric_columns_images
df_images_test = None

# dados de teste sem imagens
for column in numeric_columns_images:
    serie_all_data = pd.concat([df_train[column],df_test[column]])
    df_test[column].fillna(value=serie_all_data.median(), inplace=True)
    #df_train[column].fillna(value=serie_all_data.median(), inplace=True)
    print("Dados faltantes de coluna "+ column +" preenchidos com "+str(serie_all_data.median()))
    
print('\n')

serie_all_data = None
numeric_columns_images = None

end = time.time()

print('Tempo de merge com dataset de dados das imagens : '+str("%.2f" % (end - start))+'s\n')

#df_train.head(5)

Dados faltantes de coluna width preenchidos com 360.0
Dados faltantes de coluna height preenchidos com 360.0
Dados faltantes de coluna size preenchidos com 41615.0
Dados faltantes de coluna dullness preenchidos com 0.0
Dados faltantes de coluna whiteness preenchidos com 0.0
Dados faltantes de coluna average_red preenchidos com 0.4398931394263132
Dados faltantes de coluna average_green preenchidos com 0.4695427332970225
Dados faltantes de coluna average_blue preenchidos com 0.5056483297022513
Dados faltantes de coluna average_pixel_width preenchidos com 2.6122685185185186
Dados faltantes de coluna blurrness_score preenchidos com 554.7746435565629


Tempo de merge com dataset de dados das imagens : 13.91s



# Processando colunas geográficas   

In [6]:
# Coordenadas das cidades fornecidas pelo usuário FrankHerfert
# Ref.: https://www.kaggle.com/frankherfert/region-and-city-details-with-lat-lon-and-clusters/data

start = time.time()

# Carregando dataset e fazendo merge
df_coords = pd.read_csv('./data/avito_region_city_features.csv', encoding='utf8')
df_coords.drop(columns=["region_id","city_region_id","city_region"], inplace=True)
df_train  = df_train.merge(df_train.merge(df_coords,on = ["region","city"], copy=False, sort=False),copy=False)
df_test   = df_test.merge(df_test.merge(df_coords,on = ["region","city"], copy=False, sort=False),copy=False)

# Apgando colunas de chaves
df_train.drop (columns=["region","city"], inplace = True)
df_test.drop  (columns=["region","city"], inplace = True)
df_coords.drop(columns=["region","city"], inplace = True)

colunas_numericas_preenchidas += df_coords.columns.tolist()

# Liberando memória
df_coords = None

end = time.time()

print('Tempo de merge com dataset de dados geográficos : '+str("%.2f" % (end - start))+'s\n')

#df_train.head(5)

Tempo de merge com dataset de dados geográficos : 11.40s



# Processamento de colunas temporais

In [7]:
#Convertendo datas

start = time.time()

date_column_name = 'activation_date'
df_train[date_column_name] = df_train[date_column_name].astype('datetime64[ns]', copy=False)
df_test [date_column_name] = df_test[date_column_name].astype('datetime64[ns]', copy=False)

# Data será transforamda em três colunas: ano - 1970, mês e dia
df_train[date_column_name] = df_train[date_column_name].map(lambda t: [t.year -1970, t.month,t.day]) 
df_train['year']  = df_train[date_column_name].map(lambda t: t[0]) 
df_train['month'] = df_train[date_column_name].map(lambda t: t[1])
df_train['day']   = df_train[date_column_name].map(lambda t: t[2])
df_train.drop(columns=[date_column_name], inplace=True)

df_test[date_column_name] = df_test[date_column_name].map(lambda t: [t.year -1970, t.month,t.day]) 
df_test['year']  = df_test[date_column_name].map(lambda t: t[0]) 
df_test['month'] = df_test[date_column_name].map(lambda t: t[1])
df_test['day']   = df_test[date_column_name].map(lambda t: t[2])
df_test.drop(columns=[date_column_name], inplace=True)

numeric_columns_dates = ["year","month","day"]
colunas_numericas_preenchidas += numeric_columns_dates

end = time.time()

print('Tempo de processamento da coluna de datas : '+str("%.2f" % (end - start))+'s\n')

#df_train[numeric_columns_dates].head(5)

Tempo de processamento da coluna de datas : 4.40s



# Processamento de texto corrido

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import RussianStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.display import HTML
import string

#exibir_dataframe = lambda dataframe: display(HTML(dataframe.head(5).to_html()))

#Stop words russas
stop_words_ru = open('data/stopwords.txt', encoding='utf8').read().split('\n')
stop_words_ru = np.array(list(map(lambda x: str.lower(x),stop_words_ru)))

def stemmed_words(doc):
    stemmer = RussianStemmer()
    analyzer = CountVectorizer().build_analyzer()
    return (stemmer.stem(w) for w in analyzer(doc))

# Referência para pontuações
dict_punctuation = {}
for i in string.punctuation:
    dict_punctuation[i] = ' '
dict_punctuation = str.maketrans(dict_punctuation)

#Pré processa os dados
for column in text_columns:
    
    start = time.time()
    
    # Tratamento do texto
    # passando tudo para minúsculo
    df_train[column] = df_train[column].apply(lambda x: str.lower(str(x)))
    df_test [column] = df_test [column].apply(lambda x: str.lower(str(x)))
    # removendo pontuação
    df_train[column] = df_train[column].apply(lambda text:text.translate(dict_punctuation))
    df_test [column] = df_test [column].apply(lambda text:text.translate(dict_punctuation))
    # removendo espaços
    df_train[column] = df_train[column].apply(lambda x: str(x).strip())
    df_test [column] = df_test [column].apply(lambda x: str(x).strip())
    # removendo stop words
    df_train[column] = df_train[column].apply(lambda x: ' '.join([word.strip() for word in x.split() if word.strip() not in stop_words_ru]))
    df_test [column] = df_test [column].apply(lambda x: ' '.join([word.strip() for word in x.split() if word.strip() not in stop_words_ru]))
  
    # Criando tfIdfVectorizer
    tfidf = TfidfVectorizer(sublinear_tf=True, analyzer=stemmed_words,
                        lowercase=True, min_df = 2, max_df=0.8, dtype = np.float64)

    # fit 
    tfidf.fit(df_train[column])

    # transform
    train_column_transformed = tfidf.transform(df_train[column])
    test_column_transformed  = tfidf.transform(df_test[column])
    
    # apagando colunas antigas
    df_train.drop(columns=[column],inplace=True)
    df_test.drop (columns=[column],inplace=True)
    
    # Salvando matrizes de resultado
    matrizes_train_resultado.append(train_column_transformed)
    matrizes_test_resultado.append(test_column_transformed)
    
    end = time.time()
    
    print('Tempo de processamento da coluna '+column+' : '+str("%.2f" % (end - start))+'s\n')

#print("Coluna "+column+ " transformada em:")
#exibir_dataframe(pd.DataFrame(train_column_transformed[:3].toarray(), columns = tfidf.vocabulary_.keys()))

# Limpando memória
train_column_transformed = None
test_column_transformed  = None
tfidf = None

Tempo de processamento da coluna title : 154.87s

Tempo de processamento da coluna description : 1007.78s



# Processamento de colunas categóricas

In [9]:
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

def dropcols_coo(M, idx_to_drop):
    # Apaga coluna idx_to_drop em matriz esparsa M
    idx_to_drop = np.unique(idx_to_drop)
    C = M.tocoo()
    M = None
    keep = ~np.in1d(C.col, idx_to_drop)
    C.data, C.row, C.col = C.data[keep], C.row[keep], C.col[keep]
    C.col -= idx_to_drop.searchsorted(C.col)    
    C._shape = (C.shape[0], C.shape[1] - len(idx_to_drop))
    return C.tocsr()

num_linhas_treino = df_train.shape[0]
 
def hotEncodar(column, apagar_coluna_antiga = True):
    start = time.time()
    
    df_all_data = pd.DataFrame()
    df_all_data[column] = pd.concat([df_train[column],df_test [column]],copy=False)    
    df_all_data[column] = df_all_data[column].astype(str, copy = False)
    
     # fit labels
    label_enc = preprocessing.LabelEncoder() 
    label_enc.fit(df_all_data[column])
    
    # transform labels
    column_label_encoded = label_enc.transform(df_all_data[column]).reshape(-1,1) 
    df_all_data = None
        
    # fit e transform usando OneHotEncoder
    one_hot_enc = OneHotEncoder(dtype=np.uint8, sparse=True)
    column_transformed = one_hot_enc.fit_transform(column_label_encoded)
    
    # particionando resultado
    train_column_transformed = column_transformed[:num_linhas_treino]
    test_column_transformed  = column_transformed[num_linhas_treino:]
    column_transformed = None
    
    # removendo coluna linearmente dependente do resultado
    indice_ultima_coluna = train_column_transformed.shape[1]-1
    train_column_transformed = dropcols_coo(train_column_transformed, indice_ultima_coluna) 
    indice_ultima_coluna = test_column_transformed.shape[1]-1
    test_column_transformed = dropcols_coo(test_column_transformed, indice_ultima_coluna)
    
    # apagando colunas antigas
    if(apagar_coluna_antiga):
        df_train.drop(columns=[column], inplace=True)
        df_test.drop(columns=[column], inplace=True)
    
    # Salvando matrizes de resultado
    matrizes_train_resultado.append(train_column_transformed)
    matrizes_test_resultado.append(test_column_transformed)
    
    end = time.time()
    
    print('Tempo de processamento da coluna '+column+' : '+str("%.2f" % (end - start))+'s\n')
    
for column in colunas_categoricas_preenchidas:
    hotEncodar(column)

# Limpando memória
train_column_transformed = None
test_column_transformed  = None

Tempo de processamento da coluna user_type : 1.17s

Tempo de processamento da coluna parent_category_name : 1.33s

Tempo de processamento da coluna category_name : 1.49s

Tempo de processamento da coluna user_id : 6.24s

Tempo de processamento da coluna image_top_1 : 2.05s



# Normalização dos dados

In [10]:
from sklearn.preprocessing import MinMaxScaler

start = time.time()

df_all_data = pd.concat([df_train[colunas_numericas_preenchidas],df_test[colunas_numericas_preenchidas]],copy=False)

# Escalando colunas numéricas 
scaler = MinMaxScaler(copy=False)
scaler.fit(df_all_data[colunas_numericas_preenchidas])
train_matriz_scaled = scaler.transform(df_train[colunas_numericas_preenchidas])
test_matriz_scaled  = scaler.transform(df_test[colunas_numericas_preenchidas])
    
# Salvando matrizes de resultado
matrizes_train_resultado.append(train_matriz_scaled)
matrizes_test_resultado.append(test_matriz_scaled)

end = time.time()

print('Tempo de processamento para normalização dos dados : '+str("%.2f" % (end - start))+'s\n')

#print("Coluna numéricas transformadas em:")
#exibir_dataframe(pd.DataFrame(train_matriz_scaled[:3], columns = numeric_columns))

# liberando memória
train_matriz_scaled = None
test_matriz_scaled = None
df_all_data = None

Tempo de processamento para normalização dos dados : 0.27s



# Visualizando param_1, param_2 e param_3

In [11]:
import scipy.sparse as sps
# Gerando matrizes
matriz_train = sps.hstack(matrizes_train_resultado).tocsr()
matriz_test  = sps.hstack(matrizes_test_resultado).tocsr()

#df_test[['param_1','param_2','param_3']].head(10)

# Preenchendo param_1

In [20]:
from sklearn.ensemble import RandomForestClassifier

start = time.time()

indices_nan_test = np.array(df_test["param_1"][df_test["param_1"].isnull()].index)

df_train["param_1"] = df_train["param_1"].map(lambda x : str(x))

if(len(indices_nan_test)>0):

    clf = RandomForestClassifier(n_jobs=-1)

    clf.fit(matriz_train, df_train["param_1"])

    classes_preditas = clf.predict(matriz_test[indices_nan_test])

    df_test["param_1"].update(pd.Series(classes_preditas, index=indices_nan_test))

#hotEncoded
hotEncodar(column = "param_1", apagar_coluna_antiga = False)

# Atualizando matrizes
matriz_train = sps.hstack(matrizes_train_resultado).tocsr()
matriz_test  = sps.hstack(matrizes_test_resultado).tocsr()

end = time.time()

print('Tempo para preencher coluna param_1 : '+str("%.2f" % (end - start))+'s\n')

#df_test[['param_1','param_2','param_3']].head(10)

Tempo de processamento da coluna param_1 : 1.38s

Tempo para preencher coluna param_1 : 4.23s



# Preenchendo param_2

In [21]:
start = time.time()

classes_param_1 = df_train['param_1'].unique()

df_train["param_2"] = df_train["param_2"].map(lambda x : str(x))

for class_param_1 in classes_param_1:
    
    indices_treinamento = df_train["param_1"][df_train["param_1"] == class_param_1].index
    
    df_train_class_param_1 = df_train[df_train["param_1"] == class_param_1]
    df_test_class_param_1  = df_test [df_test ["param_1"] == class_param_1]
    
    indices_nan_test_column = np.array(df_test_class_param_1["param_2"][df_test_class_param_1["param_2"].isnull()].index)
    
    if(len(indices_nan_test_column)>0):
        
        clf = RandomForestClassifier(n_jobs=-1)
    
        clf.fit(matriz_train[indices_treinamento], df_train_class_param_1["param_2"])
    
        classes_preditas = clf.predict(matriz_test[indices_nan_test_column])

        df_test["param_2"].update(pd.Series(classes_preditas, index=indices_nan_test_column))
        

#hotEncoded
hotEncodar(column = "param_2", apagar_coluna_antiga = False)

# Atualizando matrizes
matriz_train = sps.hstack(matrizes_train_resultado).tocsr()
matriz_test  = sps.hstack(matrizes_test_resultado).tocsr()

end = time.time()

print('Tempo para preencher coluna param_2 : '+str("%.2f" % (end - start))+'s\n')

#df_test[['param_1','param_2','param_3']].head(10)

Tempo de processamento da coluna param_2 : 1.37s

Tempo para preencher coluna param_2 : 19.32s



# Preenchendo param_3

In [22]:
start = time.time()

classes_param_1 = df_train['param_2'].unique()

df_train["param_3"] = df_train["param_3"].map(lambda x : str(x))

for class_param_1 in classes_param_1:
    
    indices_treinamento = df_train["param_2"][df_train["param_2"] == class_param_1].index
    
    df_train_class_param_1 = df_train[df_train["param_2"] == class_param_1]
    df_test_class_param_1  = df_test [df_test ["param_2"] == class_param_1]
    
    indices_nan_test_column = np.array(df_test_class_param_1["param_3"][df_test_class_param_1["param_3"].isnull()].index)
    
    if(len(indices_nan_test_column)>0):
        
        clf = RandomForestClassifier(n_jobs=-1)
    
        clf.fit(matriz_train[indices_treinamento], df_train_class_param_1["param_3"])
    
        classes_preditas = clf.predict(matriz_test[indices_nan_test_column])

        df_test["param_3"].update(pd.Series(classes_preditas, index=indices_nan_test_column))
        
        
#hotEncoded
hotEncodar(column = "param_3", apagar_coluna_antiga = False)

# Atualizando matrizes
matriz_train = sps.hstack(matrizes_train_resultado).tocsr()
matriz_test  = sps.hstack(matrizes_test_resultado).tocsr()

# Liberando memória
matrizes_train_resultado = None
matrizes_test_resultado  = None

end = time.time()

print('Tempo para preencher coluna param_3 : '+str("%.2f" % (end - start))+'s\n')

#df_test[['param_1','param_2','param_3']].head(10)

Tempo de processamento da coluna param_3 : 1.42s

Tempo para preencher coluna param_3 : 27.55s



# Visualizando price

In [23]:
df_test[['price']].head(10)

,price
0,NaN
1,3000.0
2,15000.0
3,4500.0
4,4900.0
5,500.0
6,20990.0
7,990.0
8,1200.0
9,400.0


# Preenchendo price

In [24]:
from sklearn.linear_model import Ridge

start = time.time()

indices_nan_test = np.array(df_test["price"][df_test["price"].isnull()].index)

if(len(indices_nan_test)>0):

    regr = Ridge(alpha=30)

    regr.fit(matriz_train, np.log(df_train["price"]+0.001))

    ypred = regr.predict(matriz_test[indices_nan_test])

    df_test["price"].update(pd.Series(np.exp(ypred), index=indices_nan_test))

#Liberando memória
df_train = None
matriz_train = None
matriz_test = None

end = time.time()

print('Tempo para preencher coluna price : '+str("%.2f" % (end - start))+'s\n')

#df_test[['price']].head(20)

Tempo para preencher coluna price : 0.68s



# Salvando dados de teste preenchidos

In [25]:
start = time.time()
df_test_antigo = pd.read_csv('./data/test.csv' , dtype = dtype, encoding='utf8').head(df_test.shape[0])

df_test_antigo["param_1"] = df_test["param_1"]
df_test_antigo["param_2"] = df_test["param_2"]
df_test_antigo["param_3"] = df_test["param_3"]
df_test_antigo["price"]   = df_test["price"]

df_test_antigo.to_csv('./data/test_preenchido.csv', encoding='utf-8', index=False)

end = time.time()

print('Tempo para salvamento dos resultados : '+str("%.2f" % (end - start))+'s\n')

Tempo para salvamento dos resultados : 20.54s

